**test**

In [1]:
import plotly.plotly as plot
import pandas as pd

Let's load some expression data from InterMine

In [51]:
from intermine.webservice import Service
service = Service("http://www.humanmine.org/humanmine/service")

query = service.new_query("AtlasExpression")
query.add_view(
    "condition", "expression", "pValue", "tStatistic", "dataSets.name"
)

Give the query constraints

In [56]:
query.add_constraint("pValue", "<=", "0.005")
query.add_constraint("gene.symbol", "=", "GATA1")

<BinaryConstraint: AtlasExpression.gene.symbol = GATA1>

This is good, but we can only iterate the data (like below). We need it in a dataframe for plotly. 
``` python
for row in query.rows(start=0,size=10):
    print (row["condition"], row["gene.primaryIdentifier"], row["gene.symbol"], row["gene.name"], 
        row["expression"], row["pValue"], row["tStatistic"], row["dataSets.name"])
```

Load iterator into data frame by popping it in a dict first (thanks nupur)

In [57]:
#Get data in the right shape
dict = {}
for row in query.rows(start=0,size=30):
    rowDict = row.to_d()
    dict.update({row["condition"] : rowDict})

#Now pop it into the data frame
df = pd.DataFrame(data=dict)

#The columns were the wrong way around
df = df.transpose()

Now, let's use plotly to make a graph

In [54]:
from plotly.graph_objs import *

data = [Bar(x=df["AtlasExpression.condition"],
            y=df["AtlasExpression.tStatistic"])]

plot.iplot(data, filename='basic_bar')